In [180]:
import numpy as np
from typing import List, Callable, Any
from math import pow, exp
import dataclasses
from numpy.testing import assert_allclose
from numpy import polynomial as poly

# Основные функции

In [181]:
def bisection(fun: Callable[[float], float], left_bound: float, right_bound: float, eps: float) -> float:
    assert left_bound <= right_bound
    if abs(fun(right_bound)) < eps:
        return right_bound
    if abs(fun(left_bound)) < eps:
        return left_bound
    while right_bound - left_bound > eps:
        mid = (right_bound + left_bound) / 2
        if fun(mid) * fun(right_bound) > 0:
            right_bound = mid
        else:
            left_bound = mid
    return (left_bound + right_bound) / 2

In [182]:
def newton(fun: Callable[[float], float], deriv: Callable[[float], float], x_start: float, iters: int) -> float:
    x = x_start
    for i in range(iters):
        x = x - fun(x) / deriv(x)
    return x

In [183]:
def ternary_search(fun: Callable[[float], float], left: float, right: float, eps: float) -> float:
    while right - left > eps:
        tl = (2 * left + right) / 3
        tr = (left + 2 * right) / 3
        if fun(tr) > fun(tl):
            right = tr
        else:
            left = tl
    return (left + right) / 2

# Задание 1
a) Найти  $\sqrt[k]{a}$ ($k \in \mathbb{N}, a \in \mathbb{R}$) методом бисекции.
Для этого нам необходимо найти корень уравнения $x^k-a=0$. 
Разберем несколько случаев.
1. $k$ четно, $a < 0$. Решений нет.
2. $k$ нечетно, $a < 0$. Решим задачу для модуля $a$ и умножим на -1.
3. $a \ge 0$. В качестве левой границы бисекции берем 0, в качестве правой $max(a, 1)$.

In [184]:
def kroot_bisection(a: float, k: int) -> float:
    assert k >= 1
    assert k % 2 == 1 or a >= 0
    ans =  bisection(lambda x: pow(x, k) - abs(a), 0, max(abs(a), 1), 0.001)
    if a > 0:
        return ans
    return -ans

б) методом Ньютона. Та же логика, что в прошлом пункте

In [185]:
def kroot_newton(a: float, k: int) -> float:
    assert k >= 1
    assert k % 2 == 1 or a >= 0
    ans = newton(lambda x: pow(x, k) - abs(a), lambda x: k * pow(x, k - 1), 0.5, 100)
    if a > 0:
        return ans
    else:
        return -ans

In [186]:
@dataclasses.dataclass
class KthRootCase:
    a: float
    k: int
    ans: float


KTH_ROOT_TEST_CASES = [
    KthRootCase(a = -1, k = 5, ans = -1),
    KthRootCase(a = 10, k = 3, ans = 10 ** (1 / 3)),
    KthRootCase(a = 256, k = 8, ans = 2),
    KthRootCase(a = 0.0625, k = 2, ans = 0.25)
]

In [187]:
for t in KTH_ROOT_TEST_CASES:
    assert_allclose(kroot_bisection(t.a, t.k), t.ans, atol=0.001)

In [188]:
for t in KTH_ROOT_TEST_CASES:
    assert_allclose(kroot_newton(t.a, t.k), t.ans, atol=0.001)

# Задание 2
Дан многочлен. Нужно локализовать его корни, используя эти знания, посчитать сами корни и найти глобальный минимум многочлена.
наш алгоритм будет устроен следующим образом:

1. Сначала мы посчитаем точки экстремума.
2. Далее понятно, что корни будут лежать между переменами знаков в последовательности левая граница отрезка поиска, минимальный экстремум, второй экстремум, ..., максимальный экстремум, вторая граница. (Тут надо быть аккуратнее с кратными корнями, но они сами и будут экстремумами).

После локализации сами значения легко найти бисекцией.

Глобальный минимум найдем, взяв минимум во всех экстремумах и границах отрезка.

In [189]:
def poly_roots_bounds(poly, left: float, right: float, eps: float):
    roots = []
    extremums = [left]
    extremums.extend(poly_roots(poly.deriv(), left, right, eps))
    extremums.append(right)
    ans = []
    for i in range(1, len(extremums)):
        if poly(extremums[i - 1]) * poly(extremums[i]) <= 0:
            ans.append((extremums[i - 1], extremums[i]))
    for i in extremums:
        if abs(poly(i)) < eps:
            ans.append((i, i))
    return ans
    
def poly_roots(poly, left: float, right: float, eps: float):
    if poly.degree() == 0:
        return []
    bounds = poly_roots_bounds(poly, left, right, eps)
    ans = []
    for (tl, tr) in bounds:
        ans.append(bisection(poly, tl, tr, eps))
    return sorted(set(ans))

def poly_min(poly, left: float, right: float):
    extremums = [left]
    extremums.extend(poly_roots(poly.deriv(), left, right, 0.001))
    extremums.append(right)
    ans = poly(left)
    for x in extremums:
        ans = min(ans, poly(x))
    return ans

In [190]:
@dataclasses.dataclass
class Task2Case:
    poly: Any
    left: float
    right: float
    roots: List[float]
    min_val: float


TASK2_TEST_CASES = [
    Task2Case(poly=poly.Polynomial(coef=(1,2)), left=-100, right=100, roots=[-0.5], min_val=-199),
    Task2Case(poly=poly.Polynomial(coef=(-1, 1)) ** 3, left=-1, right = 2, roots=[1],min_val=-8),
    Task2Case(poly=poly.Polynomial(coef=(-10, 1)) * poly.Polynomial(coef=(-5, 1)) ** 2 * poly.Polynomial(coef=(1, 1)),
              left=0, right=10, roots=[5, 10], min_val=-292.40385)
]

In [191]:
for t in TASK2_TEST_CASES:
    assert_allclose(poly_roots(t.poly, t.left, t.right, 0.001), t.roots, atol=0.001)

In [192]:
for t in TASK2_TEST_CASES:
    assert_allclose(poly_min(t.poly, t.left, t.right), t.min_val, atol=0.001)

# Задание 3
Найти минимум функции $f(x) = e^{ax} + e^{-bx} + c(x - d)^2$. Заметим, что эта функция на плюс и минус бесконечностях стремится к плюс бесконечности, она также непрерывная, а значит, нам надо найти 0 ее производной (там и будет минимум). Это, кажется, теорема Вейерштрасса.

$f'(x) = ae^{ax}-be^{-bx}+2c(x-d)$. Она на минус бесконечности уходит в минус бесконечность, а на плюс бесконечности в плюс бесконечность. Значит, хотя бы один корень у нее есть и его можно будет найти методом бисекции.

$f''(x) = a^2 e^{ax} + b^2 e^{-bx} + 2c$. Т.к. все коэффициенты положительны, данная функция строго больше нуля, а значит, первая производная строго возрастает и имеет ровно один ноль, что значительно упрощает нам задачу. Это так же сообщает нам о том, что мы можем использовать метод тернарного поиска для поиска минимума исходной функции, так как она будет унимодальная (из производной видим, что исходная функция сначала убывает до минимума, а потом возрастает).

а) Решить данную задачу методом бисекции. Для этого нам нужно найти такие $left$ и $right$, что $f'(left) < 0$, $f'(right) > 0$. Тут нам поможет факт о том, что на минус бесконечности значение уходит в минус бесконечность, а на плюс -- в плюс. Будем следовать следующему алгоритму.
1. Установим left = -1, right = 1
2. Пока f(right) < 0, right *= 2
3. Пока f(left) > 0, left *= 2

Аналогично мы найдем границы для тернарного поиска.

In [193]:
def find_bounds(fun: Callable[[float], float]) -> (float, float):
    left = -1
    right = 1
    while fun(left) > 0:
        left *= 2
    while fun(right) < 0:
        right *= 2
    return (left, right)

In [194]:
def task3_bisection(a: float, b: float, c: float, d: float, eps: float) -> float:
    df = lambda x: a * exp(a * x) - b * exp(-b * x) + 2 * c * (x - d)
    left_bound, right_bound = find_bounds(df)
    return bisection(df, left_bound, right_bound, eps)

In [195]:
def task3_newton(a: float, b: float, c: float, d: float, iters: int) -> float:
    df = lambda x: a * exp(a * x) - b * exp(-b * x) + 2 * c * (x - d)
    ddf = lambda x: a ** 2 * exp(a * x) + b ** 2 * exp(-b * x) + 2 * c
    return newton(df, ddf, 0, iters)

In [196]:
def task3_ternary_search(a: float, b: float, c: float, d: float, eps: float) -> float:
    df = lambda x: a * exp(a * x) - b * exp(-b * x) + 2 * c * (x - d)
    left_bound, right_bound = find_bounds(df)
    f = lambda x: exp(a * x) + exp(-b * x) + c * (x - d) ** 2
    return ternary_search(f, left_bound, right_bound, eps)

In [197]:
@dataclasses.dataclass
class Task3Case:
    a: float
    b: float
    c: float
    d: float
    ans: float


TASK3_TEST_CASES = [
    Task3Case(a=1, b=1, c=1, d=1, ans=0.4901),
    Task3Case(a=1, b=5, c=1, d=1, ans=0.4595),
    Task3Case(a=0.5, b=1, c=0.1, d=10, ans=2.3557)
]

In [198]:
for t in TASK3_TEST_CASES:
    assert_allclose(task3_bisection(t.a, t.b, t.c, t.d, 0.001), t.ans, atol=0.001)

In [199]:
for t in TASK3_TEST_CASES:
    assert_allclose(task3_newton(t.a, t.b, t.c, t.d, 100), t.ans, atol=0.001)

In [200]:
for t in TASK3_TEST_CASES:
    assert_allclose(task3_ternary_search(t.a, t.b, t.c, t.d, 0.001), t.ans, atol=0.001)